# **IMPORTING LIBRARIES**

In [1]:
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import numpy as np
import pandas as pd
import csv
import joblib
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from tensorflow.keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import accuracy_score, confusion_matrix
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')
tf.get_logger().setLevel('INFO')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Mounted at /content/drive


# **DATA SANITY**

In [ ]:
#Loading Data
dataset = pd.read_csv(r'C:\Users\ayush.bajpayee\Desktop\Amazon Hackathon\dataset\train.csv', escapechar='\\',
                      quoting = csv.QUOTE_NONE)

In [ ]:
#Do not Tamper this df_original
df_original = dataset.copy()

In [ ]:
# #DO NOT RUN
# dataset_to_be_saved = dataset.copy()
# dataset_to_be_saved.to_csv('ModifiedTRAIN.csv', index = False)

In [ ]:
dataset.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4


In [ ]:
dataset.shape

(2903024, 5)

In [ ]:
dataset.columns

Index(['TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'BRAND', 'BROWSE_NODE_ID'], dtype='object')

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2903024 entries, 0 to 2903023
Data columns (total 5 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   TITLE           object
 1   DESCRIPTION     object
 2   BULLET_POINTS   object
 3   BRAND           object
 4   BROWSE_NODE_ID  int64 
dtypes: int64(1), object(4)
memory usage: 110.7+ MB


In [ ]:
dataset.isna().mean()

TITLE             0.000024
DESCRIPTION       0.249279
BULLET_POINTS     0.057272
BRAND             0.019544
BROWSE_NODE_ID    0.000000
dtype: float64

In [ ]:
dataset.isna().sum()

TITLE                 71
DESCRIPTION       723664
BULLET_POINTS     166263
BRAND              56737
BROWSE_NODE_ID         0
dtype: int64

In [ ]:
print(dataset['TITLE'][0])
print('\n')
print(dataset['TITLE'][1])
print('\n')
print(dataset['TITLE'][2])
print('\n')
print(dataset['TITLE'][3])
print('\n')
print(dataset['TITLE'][4])

Pete The Cat Bedtime Blues Doll, 14.5 Inch


The New Yorker NYHM014 Refrigerator Magnet, 2 x 3.5


The Ultimate Self-Sufficiency Handbook: A Complete Guide to Baking, Crafts, Gardening, Preserving Your Harvest, Raising Animals and More


Amway Nutrilite Kids Chewable Iron Tablets (100)


Teacher Planner Company A4 6 Lesson Academic Teacher Planner - Leatherette Pink


In [ ]:
print(dataset['BULLET_POINTS'][0])
print('\n')
print(dataset['BULLET_POINTS'][1])
print('\n')
print(dataset['BULLET_POINTS'][2])
print('\n')
print(dataset['BULLET_POINTS'][3])
print('\n')
print(dataset['BULLET_POINTS'][4])
print('\n')
print(dataset['BULLET_POINTS'][5])

[Pete the Cat Bedtime Blues plush doll,Based on the popular Pete the Cat books by James Dean,Super cuddly and ready for naptime and bedtime,Safe for all ages, perfect for ages 2 and up,Measures 14. 5 inches]


[Cat In A Tea Cup by New Yorker cover artist Gurbuz Dogan Eksioglu,Handsome cello wrapped hard magnet,Ideal for home or office, or for a gift and for any New Yorker magazine lover,It highlight one of the many beautiful New Yorker covers in full color,Rigid magnet measures 2. 5-Inch width by 3. 5-Inch height]


Skyhorse Publishing


[Nutrilite Kids,Chewable Iron Tablets,Quantity 100 tablets]


nan


[Color: Blue,Sleeve: Full Sleeve,Material: Cotton,Neck: Round]


In [ ]:
dataset['BULLET_POINTS'].apply(type).value_counts()

<class 'str'>      2736761
<class 'float'>     166263
Name: BULLET_POINTS, dtype: int64

In [ ]:
dataset_onlyfloat = dataset.copy()

In [ ]:
dataset_onlyfloat['BULLET_POINTS'].fillna(value = 'OTHERS', inplace=True)

In [ ]:
dataset_onlyfloat.isna().sum()

TITLE                 71
DESCRIPTION       723664
BULLET_POINTS          0
BRAND              56737
BROWSE_NODE_ID         0
dtype: int64

In [ ]:
dataset_onlyfloat[dataset_onlyfloat['BULLET_POINTS'].notna()]

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",Bhavya Enterprise,5
...,...,...,...,...,...
2903018,The NXT LVL Liquid Silicone Soft Back Cover Ca...,Product color may slightly vary due to photogr...,[Liquid silicone case compatible with apple pr...,Generic,794481
2903019,Premium Aviator Sunglasses - HD Polarized (Bri...,These premium Aviator Sunglasses with 5 color ...,"[Frame size: Lens height - 56mm, Lens width - ...",Generic,1040
2903020,Social Distance Stickers - Set of 5 Sticker Sl...,set of 5 prints social distancing sticker self...,[covid19 safety sticker - set of 5 to maintain...,Generic,15199
2903021,Torr-to Face Shield PACK OF 5 with Adjustable ...,* COMPLETE FACE PROTECTION: Torr-to Face Shiel...,"[350 MICRONS PACK OF 5 PCS,COMPLETE FACE PROTE...",TORR-TO,1044933


In [ ]:
dataset_onlyfloat[dataset_onlyfloat['BULLET_POINTS'].apply(type) == type('csdv')]

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",Bhavya Enterprise,5
...,...,...,...,...,...
2903018,The NXT LVL Liquid Silicone Soft Back Cover Ca...,Product color may slightly vary due to photogr...,[Liquid silicone case compatible with apple pr...,Generic,794481
2903019,Premium Aviator Sunglasses - HD Polarized (Bri...,These premium Aviator Sunglasses with 5 color ...,"[Frame size: Lens height - 56mm, Lens width - ...",Generic,1040
2903020,Social Distance Stickers - Set of 5 Sticker Sl...,set of 5 prints social distancing sticker self...,[covid19 safety sticker - set of 5 to maintain...,Generic,15199
2903021,Torr-to Face Shield PACK OF 5 with Adjustable ...,* COMPLETE FACE PROTECTION: Torr-to Face Shiel...,"[350 MICRONS PACK OF 5 PCS,COMPLETE FACE PROTE...",TORR-TO,1044933


In [ ]:
dataset['BRAND'].apply(type).value_counts()

<class 'str'>      2846287
<class 'float'>      56737
Name: BRAND, dtype: int64

# **DATA PREPROCESSING**

## Dropping Description

In [ ]:
dataset.drop(columns='DESCRIPTION', inplace = True)

## Handling Duplicates

In [ ]:
#Handling Duplicates
dataset.drop_duplicates(inplace=True)

## Handling **Nulls**

All nulls in all columns, will be replaced by **'OTHERS'**

In [ ]:
#Filling Nulls with OTHERS
dataset.fillna('OTHERS', inplace=True)

In [ ]:
dataset.isna().sum()

## Handling **BULLET_POINTS**

BULLET_POINTS contains a string either **containing** '[]' denoting it is a list of strings or **not containing** '[]' denoting there is only one Bullet Point. Will convert Bullet Points into That specific format

In [ ]:
print(dataset['BULLET_POINTS'][0])
print('\n')
print(dataset['BULLET_POINTS'][1])
print('\n')
print(dataset['BULLET_POINTS'][2])
print('\n')
print(dataset['BULLET_POINTS'][3])
print('\n')
print(dataset['BULLET_POINTS'][4])
print('\n')
print(dataset['BULLET_POINTS'][5])

In [ ]:
#Replacing ] and [ with ''
dataset['BULLET_POINTS'] = dataset['BULLET_POINTS'].str.replace('[', '')
dataset['BULLET_POINTS'] = dataset['BULLET_POINTS'].str.replace(']', '')

In [ ]:
print(dataset['BULLET_POINTS'][0])
print('\n')
print(dataset['BULLET_POINTS'][1])
print('\n')
print(dataset['BULLET_POINTS'][2])
print('\n')
print(dataset['BULLET_POINTS'][3])
print('\n')
print(dataset['BULLET_POINTS'][4])
print('\n')
print(dataset['BULLET_POINTS'][5])

In [ ]:
#Converting BULLET_POINTS from comma seperated values to list containing values
dataset['BULLET_POINTS'] = dataset['BULLET_POINTS'].map(lambda x: x.split(','))

In [ ]:
dataset['BULLET_POINTS'].apply(type).value_counts()

## Dumping/Loading

In [ ]:
#Dumping
joblib.dump(dataset, 'V1p37BIG', compress='zlib')

In [ ]:
#Loading
dataset = joblib.load('V1')

In [ ]:
dataset_original = dataset.copy()
dataset_temp = dataset.iloc[:10000,:].copy()

# **NLTK**

## Preprocessing

In [ ]:
# Tokenize
dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(word_tokenize)

In [ ]:
def remove_punc(x):
    x = [''.join(c for c in s if c not in string.punctuation) for s in x]
    x = [s for s in x if s]
    return x

dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(remove_punc)

C:\Users\AYUSH~1.BAJ\AppData\Local\Temp/ipykernel_4940/4247752062.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(remove_punc)


In [ ]:
dataset_temp

,TITLE,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","[Pete the Cat Bedtime Blues plush doll, Based ...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,[Skyhorse Publishing],imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),"[Nutrilite Kids, Chewable Iron Tablets, Quanti...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,[OTHERS],OTHERS,4
...,...,...,...,...
9998,Brass Tabla Set Brass Material … (Hammered Bra...,[Brass Bayan (Bass - Left Metal Drum) Head 9 t...,Fasherati,2603
9999,Max Men's Slim Fit Jeans (WI18CDGV08_Mid Blue_28),"[Color Name: Mid Blue, Material: Cotton, Zip f...",Max,40
10000,Gems Gallery Natural & Pure Stone Crystal Bead...,[Rare and Precious gemstone like rose quartz a...,Gems Gallery,1010
10001,Max Men's Checkered Slim Fit Formal Shirt (FBW...,"[Color Name: White, Material: Cotton, Long sle...",Max,1254


In [ ]:
X = dataset_temp['TITLE']
y = dataset_temp['BROWSE_NODE_ID']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [ ]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
# print(classification_report(y_test, y_pred))

accuracy 0.138
Wall time: 548 ms


In [ ]:
dataset_temp = dataset.iloc[:100000,:].copy()
dataset_temp.shape

(100000, 4)

In [ ]:
X = dataset_temp['TITLE']
y = dataset_temp['BROWSE_NODE_ID']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
# print(classification_report(y_test, y_pred))

MemoryError: Unable to allocate 2.63 GiB for an array with shape (353640000,) and data type float64

## Basic Approach

In [ ]:
dataset_temp = dataset.iloc[:1000000,:].copy()
dataset_temp

,TITLE,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","[Pete the Cat Bedtime Blues plush doll, Based ...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,[Skyhorse Publishing],imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),"[Nutrilite Kids, Chewable Iron Tablets, Quanti...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,[OTHERS],OTHERS,4
...,...,...,...,...
1019467,Macmerise Metallic Captain America Printed Bac...,"[Ultra vivid, razor sharp colour print techno...",Macmerise,1045
1019468,KD Willmax Stocking Over Knee Soccer Basket Ba...,"[FABRIC, WELL STITCHED and FINISHED: The spor...",KD,27627
1019469,COVERNEW Flip cover for Microsoft Nokia Lumia ...,[Durable Colour],COVERNEW,1295
1019470,DDR3 4 Gb Ram For Gigabyte G1.Sniper 3 (rev. 1...,"[4 Gb DDR 3 Desktop Ram, Un-buffered, Non-ECC...",Bright Solutions,165


In [ ]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(dataset_temp['TITLE'], dataset_temp['BROWSE_NODE_ID'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

MemoryError: Unable to allocate 50.0 GiB for an array with shape (750000, 8945) and data type int64

In [ ]:
#Transforming Test
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [ ]:
%%time

#Train Accuracy
y_pred = clf.predict(X_train_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_train))

accuracy 0.25093333333333334
Wall time: 53.3 s


In [ ]:
%%time

#Test Accuracy
y_pred = clf.predict(X_test_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_test))

accuracy 0.2306
Wall time: 15.9 s


## Further with Stemming

In [ ]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
#Porter Stemmer
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
sno = nltk.stem.SnowballStemmer('english')
# stemmed = [porter.stem(word) for word in tokens]

In [ ]:
dataset_temp = dataset.iloc[:100000,:].copy()

In [ ]:
#PORTER STEMMING
dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(porter.stem)

In [ ]:
#Snowball STEMMING
dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(sno.stem)

In [ ]:
# dataset_temp['TITLE'].apply(tokenizer.tokenize)
dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(tokenizer.tokenize)

In [ ]:
# dataset_temp['TITLE'].apply(lambda x: [item for item in x if item not in stop_words])
dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:
%%time
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(dataset_temp['TITLE'], dataset_temp['BROWSE_NODE_ID'], random_state = 0)
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}', stop_words = 'english')
# count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(X_train)
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_counts, y_train)

MemoryError: Unable to allocate 50.0 GiB for an array with shape (750000, 8945) and data type int64

In [ ]:
%%time

#Transforming Test
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [ ]:
%%time

#Train Accuracy
y_pred = clf.predict(X_train_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_train))

In [ ]:
%%time

#Test Accuracy
y_pred = clf.predict(X_test_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_test))

# **DATA TRIM DOWN**

In [ ]:
sum(dataset['BROWSE_NODE_ID'].value_counts().head(30))

838949

In [ ]:
mapper = dataset['BROWSE_NODE_ID'].value_counts().head(30).keys()
mapper

In [ ]:
dataset_original = dataset.copy()

In [ ]:
dataset = dataset.loc[dataset['BROWSE_NODE_ID'].isin(mapper)]
dataset

,TITLE,BULLET_POINTS,BRAND,BROWSE_NODE_ID
4,Teacher Planner Company A4 6 Lesson Academic T...,OTHERS,OTHERS,4
5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",Bhavya Enterprise,5
11,White : Braun KF400-WH Aromaster 10-Cup Coffee...,"[Water-dispersal system provides fast, uniform...",Braun,11
40,Wrangler Men's Regular Fit Jeans (13MWZPW1-463...,DURABLE MATERIALS. It's no question that this...,Wrangler,40
47,Susan Howatch,OTHERS,OTHERS,4
...,...,...,...,...
2902995,Think Tech Desinger Printed Back Case Cover Co...,[It'S 3D Printing Technologies So Colour Will ...,Think Tech,1045
2903009,Pawan Textiles Womens Cotton Jaipuri Block Pri...,[Hand Block Print Jaipuri Sanganeri Salwar Sui...,Pawan Textiles,465
2903015,CRONEX Presents 100% Waterproof Car Body Cover...,[✔Compatible For-Maruti Suzuki Ignis including...,CRONEX,1590
2903016,Holy Heart Collections Supremo Zig Zag Line Pr...,"[Color : Multicolor, Material : Fine Polyester...",Holy Heart Collections,739


# **DEEP LEARNING**

In [ ]:
embeddings_index = {}
try:
    f = open('glove.840B.300d.txt', encoding='utf8')
    for line in tqdm(f):
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))
    
except:
    pass

In [ ]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [ ]:
%%time
dataset_temp = dataset.iloc[:50000,:].copy()
dataset_temp

y = dataset_temp['BROWSE_NODE_ID'].values

y_enc = to_categorical(y.copy())

#DO not tamper
X_train, X_test, y_train_will_check, y_test_will_check = train_test_split(dataset_temp['TITLE'], y.copy(), shuffle = False, random_state = 0)

X_train, X_test, y_train, y_test = train_test_split(dataset_temp['TITLE'], y_enc, shuffle = False, random_state = 0)

xtrain_glove = [sent2vec(x) for x in tqdm(X_train.copy())]
xtest_glove = [sent2vec(x) for x in tqdm(X_test.copy())]

xtrain_glove = np.array(xtrain_glove)
xtest_glove = np.array(xtest_glove)

scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xtest_glove_scl = scl.transform(xtest_glove)

model = Sequential()
print('Model Initialised')

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
print('3 layers added')

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
print('3 layers added')

model.add(Dense(y_enc.shape[1]))
model.add(Activation('softmax'))
print('Softmax added')

# compile the model
print('Model compiling')
model.compile(loss='categorical_crossentropy', optimizer='adam')
print('DONE')

model.fit(xtrain_glove_scl, y=y_train, batch_size=20, 
          epochs=5, verbose=1, 
          validation_data=(xtest_glove_scl, y_test))

y_pred = model.predict_classes(xtest_glove_scl)

# #Test Accuracy
# y_pred = clf.predict(X_test_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_test_will_check))

MemoryError: Unable to allocate 329. MiB for an array with shape (50000, 1725) and data type float32

In [ ]:
#For batch size: 20
accuracy 0.3336
Wall time: 1min 35s

# **CONVERSION OF DATAFRAME TO TENSOR**

In [ ]:
dataset

In [ ]:
tempTensor = tf.convert_to_tensor(dataset['TITLE'].copy())

In [ ]:
tempTensor

In [ ]:
list(dataset['TITLE'].map(lambda x: x.split(',')))

In [ ]:
list(dataset['TITLE'])

In [ ]:
list(tempTensor.numpy())

In [ ]:
dataset['TITLE'].tolist()

# **Using Title as a Feature**

In [3]:
!pip install transformers
dataset = joblib.load('/content/drive/MyDrive/pickles/8lakhrows')
dataset_original = dataset.copy()
dataset = dataset_original.iloc[:500000, :]

     |████████████████████████████████| 2.6 MB 8.3 MB/s 
     |████████████████████████████████| 3.3 MB 58.1 MB/s 
     |████████████████████████████████| 895 kB 69.2 MB/s 
     |████████████████████████████████| 636 kB 60.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from sklearn.model_selection import train_test_split

X = dataset['TITLE'].values   # independant features
y = dataset['BROWSE_NODE_ID'].values					# dependant variable

# Choose your test size to split between training and testing sets:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle = False)
# X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, shuffle = False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify = y)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify = y_test)

In [5]:
model_name = "distilbert-base-uncased"
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

In [6]:
# Define the maximum number of words to tokenize (DistilBERT can tokenize up to 512)
MAX_LENGTH = 32


# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=12800, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts'
    corresponding encodings and attention masks that are ready to be fed 
    into a pre-trained transformer model.
    
    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""
    
    input_ids = []
    attention_mask = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='longest', #implements dynamic padding
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    # return tf.ragged.constant(input_ids), tf.ragged.constant(attention_mask)
    
    
# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.tolist())

# Encode X_valid
X_valid_ids, X_valid_attention = batch_encode(tokenizer, X_valid.tolist())

# Encode X_test
X_test_ids, X_test_attention = batch_encode(tokenizer, X_test.tolist())

In [7]:
from transformers import TFDistilBertModel, DistilBertConfig

DISTILBERT_DROPOUT = 0.2
DISTILBERT_ATT_DROPOUT = 0.2
 
# Configure DistilBERT's initialization
config = DistilBertConfig(dropout=DISTILBERT_DROPOUT, 
                          attention_dropout=DISTILBERT_ATT_DROPOUT, 
                          output_hidden_states=True)
                          
# The bare, pre-trained DistilBERT transformer model outputting raw hidden-states 
# and without any specific head on top.
distilBERT = TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)

# Make DistilBERT layers untrainable
for layer in distilBERT.layers:
    layer.trainable = False

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [8]:
MAX_LENGTH = 32
LAYER_DROPOUT = 0.2
LEARNING_RATE = 5e-5
RANDOM_STATE = 42
transformer = distilBERT

def build_model(transformer, max_length=MAX_LENGTH):
    # """""""""
    # Template for building a model off of the BERT or DistilBERT architecture
    # for a binary classification task.
    
    # Input:
    #   - transformer:  a base Hugging Face transformer model object (BERT or DistilBERT)
    #                   with no added classification head attached.
    #   - max_length:   integer controlling the maximum number of encoded tokens 
    #                   in a given sequence.
    
    # Output:
    #   - model:        a compiled tf.keras.Model with added classification layers 
    #                   on top of the base pre-trained model architecture.
    # """"""""""
    
    # Define weight initializer with a random seed to ensure reproducibility
    # weight_initializer = tf.keras.initializers.GlorotNormal(seed=RANDOM_STATE) 
    
    # Define input layers
    input_ids_layer = tf.keras.layers.Input(shape=(max_length,), 
                                            name='input_ids', 
                                            dtype='int32')
    input_attention_layer = tf.keras.layers.Input(shape=(max_length,), 
                                                  name='input_attention', 
                                                  dtype='int32')
    
    # DistilBERT outputs a tuple where the first element at index 0
    # represents the hidden-state at the output of the model's last layer.
    # It is a tf.Tensor of shape (batch_size, sequence_length, hidden_size=768).
    last_hidden_state = transformer([input_ids_layer, input_attention_layer])[0]
    
    # We only care about DistilBERT's output for the [CLS] token, 
    # which is located at index 0 of every encoded sequence.  
    # Splicing out the [CLS] tokens gives us 2D data.
    cls_token = last_hidden_state[:, 0, :]
    
    ##                                                 ##
    ## Define additional dropout and dense layers here ##
    ##                                                 ##
    
    # Define a single node that makes up the output layer (for binary classification)
    # model.add(Dense(y_enc.shape[1]))
    # model.add(Activation('softmax'))
    # outputs = keras.layers.Dense(30,
    #                          kernel_initializer=initializer,
    #                          activation=activation)(x) 
    output = tf.keras.layers.Dense(30, 
                                   activation='softmax',
                                  #  kernel_initializer=weight_initializer,  
                                  #  kernel_constraint=None,
                                   bias_initializer='zeros'
                                   )(cls_token)
    
    # Define the model
    model = tf.keras.Model([input_ids_layer, input_attention_layer], output)
    # model = tf.keras.Model([input_ids_layer, input_attention_layer])
    # model.add(Dense(30, activation='softmax'))
    
    # Compile the model
    model.compile(tf.keras.optimizers.Adam(lr=LEARNING_RATE), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [9]:
#Building Model
model = build_model(transformer, MAX_LENGTH)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [23]:
# y_train = to_categorical(pd.Series(y_train), 30)
# y_test = to_categorical(pd.Series(y_test), 30)
temp2 = pd.get_dummies(pd.Series(y_train.copy()))

In [ ]:
pd.Series(y_train).nunique()

30

In [10]:
EPOCHS = 20
BATCH_SIZE = 512
NUM_STEPS = len(X_train) // BATCH_SIZE

# Train the model
train_history1 = model.fit(
    x = [X_train_ids, X_train_attention],
    y = np.array(pd.get_dummies(pd.Series(y_train))),
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    steps_per_epoch = NUM_STEPS,
    validation_data = ([X_valid_ids, X_valid_attention], np.array(pd.get_dummies(y_valid))),
    verbose=1
)

Epoch 1/20
732/732 [==============================] - 468s 631ms/step - loss: 2.7812 - accuracy: 0.2353 - val_loss: 2.3059 - val_accuracy: 0.3144
Epoch 2/20
732/732 [==============================] - 465s 635ms/step - loss: 2.1593 - accuracy: 0.3922 - val_loss: 1.8209 - val_accuracy: 0.4916
Epoch 3/20
732/732 [==============================] - 463s 633ms/step - loss: 1.7928 - accuracy: 0.5196 - val_loss: 1.5039 - val_accuracy: 0.6091
Epoch 4/20
732/732 [==============================] - 463s 632ms/step - loss: 1.5399 - accuracy: 0.6000 - val_loss: 1.2865 - val_accuracy: 0.6752
Epoch 5/20
732/732 [==============================] - 464s 633ms/step - loss: 1.3590 - accuracy: 0.6525 - val_loss: 1.1298 - val_accuracy: 0.7176
Epoch 6/20
732/732 [==============================] - 464s 634ms/step - loss: 1.2238 - accuracy: 0.6883 - val_loss: 1.0132 - val_accuracy: 0.7475
Epoch 7/20
732/732 [==============================] - 463s 633ms/step - loss: 1.1179 - accuracy: 0.7164 - val_loss: 0.9225 -

In [11]:
model.evaluate([X_valid_ids, X_valid_attention], np.array(pd.get_dummies(y_valid)))

1954/1954 [==============================] - 78s 40ms/step - loss: 0.5128 - accuracy: 0.8676


[0.5128031373023987, 0.8676319718360901]

In [12]:
model.evaluate([X_test_ids, X_test_attention], np.array(pd.get_dummies(y_test)))

1954/1954 [==============================] - 77s 39ms/step - loss: 0.5177 - accuracy: 0.8661


[0.5177341103553772, 0.8660799860954285]

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 32)]         0                                            
__________________________________________________________________________________________________
input_attention (InputLayer)    [(None, 32)]         0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB TFBaseModelOutput(la 66362880    input_ids[0][0]                  
                                                                 input_attention[0][0]            
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 768)          0           tf_distil_bert_model[0][7]   

In [17]:
test = pd.read_csv(r'/content/drive/MyDrive/pickles/test.csv', escapechar='\\',
                      quoting = csv.QUOTE_NONE)

In [19]:
test_features = dataset['TITLE'].values   # independant features

In [20]:
# Define the maximum number of words to tokenize (DistilBERT can tokenize up to 512)
MAX_LENGTH = 32


# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=12800, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts'
    corresponding encodings and attention masks that are ready to be fed 
    into a pre-trained transformer model.
    
    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""
    
    input_ids = []
    attention_mask = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='longest', #implements dynamic padding
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    # return tf.ragged.constant(input_ids), tf.ragged.constant(attention_mask)
    
    
# Encode X_train
test_features_ids, test_features_attention = batch_encode(tokenizer, test_features.tolist())

In [21]:
testPred = model.predict([test_features_ids, test_features_attention])

In [ ]:
encoder = {}
for i, j in enumerate(temp2.columns):
  encoder[i] = j

encoder

In [22]:
pd.DataFrame(testPred)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.478941,0.000655,0.039961,0.000136,0.018931,0.010690,0.004222,0.007097,0.012408,0.000694,0.018034,0.012765,0.011901,0.003402,0.006077,0.000094,0.018397,0.000993,0.004516,0.005948,0.012366,0.017494,0.000531,0.000713,0.000166,0.000133,0.003761,0.000228,0.296882,0.011863
1,0.000934,0.710853,0.000037,0.002401,0.000059,0.004340,0.000010,0.000002,0.000173,0.000821,0.001343,0.000107,0.000687,0.000031,0.015795,0.195585,0.000057,0.028543,0.000051,0.000040,0.000191,0.000750,0.005170,0.000261,0.025372,0.000041,0.004655,0.000010,0.000408,0.001275
2,0.068866,0.019198,0.827241,0.000888,0.001433,0.020231,0.002344,0.000494,0.000482,0.000144,0.002752,0.001435,0.004799,0.000051,0.000471,0.008463,0.002261,0.000416,0.002888,0.007450,0.000619,0.005627,0.012371,0.003613,0.000459,0.000709,0.002071,0.001920,0.000093,0.000212
3,0.000032,0.114880,0.000002,0.692850,0.000016,0.005320,0.000003,0.000002,0.000008,0.000298,0.005029,0.000005,0.001079,0.000010,0.011337,0.102316,0.000029,0.006271,0.000027,0.000005,0.000171,0.003853,0.020478,0.000534,0.030672,0.000021,0.001743,0.000060,0.000645,0.002305
4,0.931949,0.000722,0.002111,0.000330,0.004700,0.000715,0.001082,0.000326,0.000629,0.001122,0.000309,0.000709,0.000639,0.003122,0.000202,0.000195,0.000354,0.001529,0.010396,0.000565,0.012549,0.005905,0.000294,0.011891,0.000474,0.000139,0.001796,0.000030,0.001395,0.003822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0.000033,0.098355,0.000008,0.001971,0.000002,0.000826,0.000001,0.000001,0.000013,0.000725,0.000325,0.000020,0.000900,0.000008,0.005114,0.688910,0.000077,0.008591,0.000144,0.000048,0.000029,0.003031,0.006741,0.000218,0.181869,0.000059,0.000628,0.000028,0.000223,0.001104
499996,0.001196,0.001009,0.000500,0.000012,0.000104,0.000084,0.000078,0.002497,0.000254,0.000045,0.000083,0.000033,0.000083,0.000129,0.000077,0.000031,0.930403,0.000094,0.000014,0.000209,0.000024,0.000051,0.000506,0.000196,0.000003,0.061911,0.000007,0.000338,0.000014,0.000015
499997,0.000416,0.027920,0.000284,0.014792,0.000066,0.003020,0.000115,0.000085,0.000205,0.037796,0.001075,0.001080,0.484472,0.001964,0.002459,0.011750,0.001244,0.056745,0.015970,0.000370,0.013672,0.128276,0.002695,0.084220,0.005166,0.001332,0.003190,0.000148,0.012946,0.086528
499998,0.000963,0.385930,0.002250,0.000126,0.000280,0.000352,0.000629,0.000080,0.020099,0.003044,0.001319,0.000047,0.000671,0.000072,0.000961,0.001111,0.007613,0.527538,0.000804,0.000145,0.004061,0.014918,0.000994,0.022139,0.000048,0.000326,0.000260,0.000079,0.000223,0.002919


In [ ]:
tempPred = pd.DataFrame(pred)
tempPred

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.023923,0.132491,0.013514,0.019273,0.009867,0.021008,0.015440,0.022611,0.013162,0.014799,0.026106,0.024532,0.038803,0.015088,0.016503,0.062014,0.155990,0.031444,0.054619,0.013503,0.020925,0.028365,0.017016,0.067521,0.014472,0.027451,0.011753,0.011247,0.022361,0.054199
1,0.029876,0.105808,0.014953,0.017558,0.010855,0.020970,0.014603,0.020567,0.016121,0.019447,0.023879,0.024578,0.033095,0.024476,0.015604,0.047480,0.154025,0.035966,0.071292,0.013566,0.023584,0.031429,0.015793,0.060330,0.013575,0.025139,0.010793,0.008992,0.025865,0.069781
2,0.042637,0.089993,0.018745,0.014072,0.011415,0.021520,0.016180,0.020600,0.016186,0.012538,0.033193,0.021981,0.039027,0.025663,0.013635,0.043400,0.137569,0.038397,0.064716,0.016186,0.023958,0.033620,0.018616,0.073781,0.020728,0.034055,0.008836,0.010188,0.017991,0.060574
3,0.032842,0.159471,0.017090,0.022189,0.013670,0.020169,0.014613,0.017903,0.013779,0.009020,0.027917,0.014774,0.033562,0.010656,0.021021,0.077997,0.160430,0.040344,0.043988,0.014432,0.018923,0.029695,0.015162,0.068178,0.017847,0.027078,0.008553,0.009650,0.014091,0.024961
4,0.025785,0.033486,0.013944,0.008872,0.007692,0.014400,0.009847,0.100938,0.007413,0.007209,0.015116,0.014640,0.019630,0.008774,0.007402,0.013464,0.487447,0.008286,0.015253,0.014354,0.010892,0.019827,0.007898,0.023632,0.004859,0.055392,0.006202,0.019524,0.007319,0.010503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.022528,0.039955,0.016269,0.010084,0.014024,0.009597,0.013329,0.045796,0.011229,0.010015,0.016046,0.014588,0.016959,0.012138,0.009606,0.014919,0.525009,0.010913,0.019390,0.012515,0.013350,0.016923,0.006119,0.023438,0.005253,0.050807,0.009409,0.008070,0.008949,0.012774
24996,0.036257,0.110410,0.025822,0.021428,0.014275,0.038738,0.014351,0.016899,0.014520,0.013363,0.040965,0.024173,0.042290,0.018410,0.018509,0.035063,0.088581,0.033215,0.071423,0.018878,0.037414,0.027527,0.018129,0.078528,0.017435,0.025105,0.015158,0.010909,0.022275,0.049952
24997,0.032884,0.060733,0.020027,0.011159,0.013584,0.014256,0.015359,0.043356,0.019235,0.010682,0.017681,0.024560,0.018604,0.011356,0.011865,0.031049,0.379482,0.015584,0.032400,0.022382,0.015320,0.026363,0.009521,0.039796,0.011585,0.037784,0.009535,0.012426,0.011083,0.020351
24998,0.024319,0.040981,0.010826,0.009396,0.006350,0.016571,0.009178,0.107338,0.009787,0.009756,0.012735,0.023894,0.026807,0.009584,0.006612,0.018054,0.404725,0.015105,0.021737,0.017123,0.012708,0.016423,0.008112,0.023958,0.005657,0.085510,0.006993,0.015575,0.008682,0.015503


In [ ]:
tempPred = tempPred.rename(columns=encoder)
tempPred

,4,5,11,40,55,89,251,374,413,465,701,739,773,803,1003,1024,1045,1049,1052,1084,1186,1194,1213,1251,1254,1295,1409,1590,1687,1724
0,0.023923,0.132491,0.013514,0.019273,0.009867,0.021008,0.015440,0.022611,0.013162,0.014799,0.026106,0.024532,0.038803,0.015088,0.016503,0.062014,0.155990,0.031444,0.054619,0.013503,0.020925,0.028365,0.017016,0.067521,0.014472,0.027451,0.011753,0.011247,0.022361,0.054199
1,0.029876,0.105808,0.014953,0.017558,0.010855,0.020970,0.014603,0.020567,0.016121,0.019447,0.023879,0.024578,0.033095,0.024476,0.015604,0.047480,0.154025,0.035966,0.071292,0.013566,0.023584,0.031429,0.015793,0.060330,0.013575,0.025139,0.010793,0.008992,0.025865,0.069781
2,0.042637,0.089993,0.018745,0.014072,0.011415,0.021520,0.016180,0.020600,0.016186,0.012538,0.033193,0.021981,0.039027,0.025663,0.013635,0.043400,0.137569,0.038397,0.064716,0.016186,0.023958,0.033620,0.018616,0.073781,0.020728,0.034055,0.008836,0.010188,0.017991,0.060574
3,0.032842,0.159471,0.017090,0.022189,0.013670,0.020169,0.014613,0.017903,0.013779,0.009020,0.027917,0.014774,0.033562,0.010656,0.021021,0.077997,0.160430,0.040344,0.043988,0.014432,0.018923,0.029695,0.015162,0.068178,0.017847,0.027078,0.008553,0.009650,0.014091,0.024961
4,0.025785,0.033486,0.013944,0.008872,0.007692,0.014400,0.009847,0.100938,0.007413,0.007209,0.015116,0.014640,0.019630,0.008774,0.007402,0.013464,0.487447,0.008286,0.015253,0.014354,0.010892,0.019827,0.007898,0.023632,0.004859,0.055392,0.006202,0.019524,0.007319,0.010503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.022528,0.039955,0.016269,0.010084,0.014024,0.009597,0.013329,0.045796,0.011229,0.010015,0.016046,0.014588,0.016959,0.012138,0.009606,0.014919,0.525009,0.010913,0.019390,0.012515,0.013350,0.016923,0.006119,0.023438,0.005253,0.050807,0.009409,0.008070,0.008949,0.012774
24996,0.036257,0.110410,0.025822,0.021428,0.014275,0.038738,0.014351,0.016899,0.014520,0.013363,0.040965,0.024173,0.042290,0.018410,0.018509,0.035063,0.088581,0.033215,0.071423,0.018878,0.037414,0.027527,0.018129,0.078528,0.017435,0.025105,0.015158,0.010909,0.022275,0.049952
24997,0.032884,0.060733,0.020027,0.011159,0.013584,0.014256,0.015359,0.043356,0.019235,0.010682,0.017681,0.024560,0.018604,0.011356,0.011865,0.031049,0.379482,0.015584,0.032400,0.022382,0.015320,0.026363,0.009521,0.039796,0.011585,0.037784,0.009535,0.012426,0.011083,0.020351
24998,0.024319,0.040981,0.010826,0.009396,0.006350,0.016571,0.009178,0.107338,0.009787,0.009756,0.012735,0.023894,0.026807,0.009584,0.006612,0.018054,0.404725,0.015105,0.021737,0.017123,0.012708,0.016423,0.008112,0.023958,0.005657,0.085510,0.006993,0.015575,0.008682,0.015503


In [ ]:
print(max(tempPred.iloc[:1].values.flatten()))
print(list(tempPred.iloc[:1].values.flatten()).index(max(tempPred.iloc[:1].values.flatten())))
print(encoder[list(tempPred.iloc[:1].values.flatten()).index(max(tempPred.iloc[:1].values.flatten()))])

0.15599021
16
1045


# **Using Bullet Points as a Feature**

In [ ]:
# !pip install transformers
dataset = joblib.load('/content/drive/MyDrive/pickles/V1p37BIG')
dataset_original = dataset.copy()
# dataset = dataset_original.iloc[:200000, :]

     |████████████████████████████████| 2.6 MB 7.4 MB/s 
     |████████████████████████████████| 636 kB 49.1 MB/s 
     |████████████████████████████████| 895 kB 77.0 MB/s 
     |████████████████████████████████| 3.3 MB 60.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
dataset = dataset_original.copy()
dataset

,TITLE,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,OTHERS,OTHERS,4
...,...,...,...,...
2903019,Premium Aviator Sunglasses - HD Polarized (Bri...,"[Frame size: Lens height - 56mm, Lens width - ...",Generic,1040
2903020,Social Distance Stickers - Set of 5 Sticker Sl...,[covid19 safety sticker - set of 5 to maintain...,Generic,15199
2903021,Torr-to Face Shield PACK OF 5 with Adjustable ...,"[350 MICRONS PACK OF 5 PCS,COMPLETE FACE PROTE...",TORR-TO,1044933
2903022,Type-C to 3.5 MM for Oppo R17 Pro Type-C to 3....,"[Indian Connectors: Made for Indian sockets, t...",SHOPBELL,14790


In [ ]:
#Trimming Down Data
mapper = dataset['BROWSE_NODE_ID'].value_counts().head(50).keys()
print(len(mapper))

dataset = dataset.loc[dataset['BROWSE_NODE_ID'].isin(mapper)]
dataset

50


,TITLE,BULLET_POINTS,BRAND,BROWSE_NODE_ID
4,Teacher Planner Company A4 6 Lesson Academic T...,OTHERS,OTHERS,4
5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",Bhavya Enterprise,5
11,White : Braun KF400-WH Aromaster 10-Cup Coffee...,"[Water-dispersal system provides fast, uniform...",Braun,11
40,Wrangler Men's Regular Fit Jeans (13MWZPW1-463...,DURABLE MATERIALS. It's no question that this...,Wrangler,40
47,Susan Howatch,OTHERS,OTHERS,4
...,...,...,...,...
2903009,Pawan Textiles Womens Cotton Jaipuri Block Pri...,[Hand Block Print Jaipuri Sanganeri Salwar Sui...,Pawan Textiles,465
2903015,CRONEX Presents 100% Waterproof Car Body Cover...,[✔Compatible For-Maruti Suzuki Ignis including...,CRONEX,1590
2903016,Holy Heart Collections Supremo Zig Zag Line Pr...,"[Color : Multicolor, Material : Fine Polyester...",Holy Heart Collections,739
2903019,Premium Aviator Sunglasses - HD Polarized (Bri...,"[Frame size: Lens height - 56mm, Lens width - ...",Generic,1040


In [ ]:
dataset['BROWSE_NODE_ID'].value_counts()

In [ ]:
dataset = dataset.iloc[:500000, :]

In [ ]:
dataset['BULLET_POINTS']

4                                                     OTHERS
5          [Color: Blue,Sleeve: Full Sleeve,Material: Cot...
11         [Water-dispersal system provides fast, uniform...
40         DURABLE MATERIALS.  It's no question that this...
47                                                    OTHERS
                                 ...                        
1403882    [Our Premium designs for Moto G5 Plus printed ...
1403883    [Fabric: Cotton,Style: Regular,Neck Style: Rou...
1403886    [Full Glue 5D Tempered Glass: Innovative Desig...
1403887    [Color Name: Blue,Material : 100% Cotton,Long ...
1403890    [Protect Your Phone from Dust,Scratches and Fu...
Name: BULLET_POINTS, Length: 500000, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split

X = dataset['BULLET_POINTS'].values   # independent features
y = dataset['BROWSE_NODE_ID'].values					# dependent variable

# Choose your test size to split between training and testing sets:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle = False)
# X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, shuffle = False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify = y)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify = y_test)

In [ ]:
model_name = "distilbert-base-uncased"
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

In [ ]:
# Define the maximum number of words to tokenize (DistilBERT can tokenize up to 512)
MAX_LENGTH = 16


# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=12800, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts'
    corresponding encodings and attention masks that are ready to be fed 
    into a pre-trained transformer model.
    
    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""
    
    input_ids = []
    attention_mask = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='longest', #implements dynamic padding
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    # return tf.ragged.constant(input_ids), tf.ragged.constant(attention_mask)
    
    
# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.tolist())

# Encode X_valid
X_valid_ids, X_valid_attention = batch_encode(tokenizer, X_valid.tolist())

# Encode X_test
X_test_ids, X_test_attention = batch_encode(tokenizer, X_test.tolist())

In [ ]:
from transformers import TFDistilBertModel, DistilBertConfig

DISTILBERT_DROPOUT = 0.2
DISTILBERT_ATT_DROPOUT = 0.2
 
# Configure DistilBERT's initialization
config = DistilBertConfig(dropout=DISTILBERT_DROPOUT, 
                          attention_dropout=DISTILBERT_ATT_DROPOUT, 
                          output_hidden_states=True)
                          
# The bare, pre-trained DistilBERT transformer model outputting raw hidden-states 
# and without any specific head on top.
distilBERT = TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)

# Make DistilBERT layers untrainable
for layer in distilBERT.layers:
    layer.trainable = False

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_transform', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
MAX_LENGTH = 16
LAYER_DROPOUT = 0.2
LEARNING_RATE = 5e-5
RANDOM_STATE = 42
transformer = distilBERT

def build_model(transformer, max_length=MAX_LENGTH):
    # """""""""
    # Template for building a model off of the BERT or DistilBERT architecture
    # for a binary classification task.
    
    # Input:
    #   - transformer:  a base Hugging Face transformer model object (BERT or DistilBERT)
    #                   with no added classification head attached.
    #   - max_length:   integer controlling the maximum number of encoded tokens 
    #                   in a given sequence.
    
    # Output:
    #   - model:        a compiled tf.keras.Model with added classification layers 
    #                   on top of the base pre-trained model architecture.
    # """"""""""
    
    # Define weight initializer with a random seed to ensure reproducibility
    # weight_initializer = tf.keras.initializers.GlorotNormal(seed=RANDOM_STATE) 
    
    # Define input layers
    input_ids_layer = tf.keras.layers.Input(shape=(max_length,), 
                                            name='input_ids', 
                                            dtype='int32')
    input_attention_layer = tf.keras.layers.Input(shape=(max_length,), 
                                                  name='input_attention', 
                                                  dtype='int32')
    
    # DistilBERT outputs a tuple where the first element at index 0
    # represents the hidden-state at the output of the model's last layer.
    # It is a tf.Tensor of shape (batch_size, sequence_length, hidden_size=768).
    last_hidden_state = transformer([input_ids_layer, input_attention_layer])[0]
    
    # We only care about DistilBERT's output for the [CLS] token, 
    # which is located at index 0 of every encoded sequence.  
    # Splicing out the [CLS] tokens gives us 2D data.
    cls_token = last_hidden_state[:, 0, :]
    
    ##                                                 ##
    ## Define additional dropout and dense layers here ##
    ##                                                 ##
    
    # Define a single node that makes up the output layer (for binary classification)
    # model.add(Dense(y_enc.shape[1]))
    # model.add(Activation('softmax'))
    # outputs = keras.layers.Dense(30,
    #                          kernel_initializer=initializer,
    #                          activation=activation)(x) 
    output = tf.keras.layers.Dense(50, 
                                   activation='softmax',
                                  #  kernel_initializer=weight_initializer,  
                                  #  kernel_constraint=None,
                                   bias_initializer='zeros'
                                   )(cls_token)
    
    # Define the model
    model = tf.keras.Model([input_ids_layer, input_attention_layer], output)
    # model = tf.keras.Model([input_ids_layer, input_attention_layer])
    # model.add(Dense(30, activation='softmax'))
    
    # Compile the model
    model.compile(tf.keras.optimizers.Adam(lr=LEARNING_RATE), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
model = build_model(transformer, MAX_LENGTH)

In [ ]:
# y_train = to_categorical(pd.Series(y_train), 30)
# y_test = to_categorical(pd.Series(y_test), 30)
temp2 = pd.get_dummies(pd.Series(y_train.copy()))

In [ ]:
encoder = {}
for i, j in enumerate(temp2.columns):
  encoder[i] = j

encoder

In [ ]:
pd.Series(y_train).nunique()

30

In [ ]:
tf.get_logger().setLevel('INFO')

In [ ]:
EPOCHS = 20
BATCH_SIZE = 1024
NUM_STEPS = len(X_train) // BATCH_SIZE

# Train the model
train_history1 = model.fit(
    x = [X_train_ids, X_train_attention],
    y = np.array(pd.get_dummies(pd.Series(y_train))),
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    steps_per_epoch = NUM_STEPS,
    validation_data = ([X_valid_ids, X_valid_attention], np.array(pd.get_dummies(y_valid))),
    verbose=1
)

Epoch 1/20
366/366 [==============================] - 229s 625ms/step - loss: 2.3962 - accuracy: 0.3813 - val_loss: 2.2894 - val_accuracy: 0.3934
Epoch 2/20
366/366 [==============================] - 232s 635ms/step - loss: 2.3536 - accuracy: 0.3913 - val_loss: 2.2460 - val_accuracy: 0.4047
Epoch 3/20
366/366 [==============================] - 233s 636ms/step - loss: 2.3157 - accuracy: 0.3998 - val_loss: 2.2076 - val_accuracy: 0.4143
Epoch 4/20
366/366 [==============================] - 233s 636ms/step - loss: 2.2803 - accuracy: 0.4082 - val_loss: 2.1728 - val_accuracy: 0.4243
Epoch 5/20
366/366 [==============================] - 233s 636ms/step - loss: 2.2501 - accuracy: 0.4155 - val_loss: 2.1405 - val_accuracy: 0.4323
Epoch 6/20
366/366 [==============================] - 233s 636ms/step - loss: 2.2212 - accuracy: 0.4228 - val_loss: 2.1114 - val_accuracy: 0.4379
Epoch 7/20
366/366 [==============================] - 233s 636ms/step - loss: 2.1948 - accuracy: 0.4291 - val_loss: 2.0841 -

In [ ]:
  model.evaluate([X_valid_ids, X_valid_attention], np.array(pd.get_dummies(y_valid)))

1954/1954 [==============================] - 46s 24ms/step - loss: 1.8665 - accuracy: 0.5056


[1.8665248155593872, 0.5056480169296265]

In [ ]:
model.evaluate([X_test_ids, X_test_attention], np.array(pd.get_dummies(y_test)))

1954/1954 [==============================] - 47s 24ms/step - loss: 1.8583 - accuracy: 0.5069


[1.8582847118377686, 0.5068960189819336]

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 16)]         0                                            
__________________________________________________________________________________________________
input_attention (InputLayer)    [(None, 16)]         0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB multiple             66362880    input_ids[0][0]                  
                                                                 input_attention[0][0]            
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 768)          0           tf_distil_bert_model[1][7] 

In [ ]:
tf.keras.models.save_model(
    model = model, filepath = '/content/drive/MyDrive/Models', overwrite=True, include_optimizer=True, save_format='tf',
    signatures=None, options=None, save_traces=False
)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Models/assets


In [ ]:
model

In [ ]:
model.predict([X_valid_ids, X_valid_attention])

StagingError: ignored

In [ ]:
tempPred = pd.DataFrame(pred)
tempPred

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.023923,0.132491,0.013514,0.019273,0.009867,0.021008,0.015440,0.022611,0.013162,0.014799,0.026106,0.024532,0.038803,0.015088,0.016503,0.062014,0.155990,0.031444,0.054619,0.013503,0.020925,0.028365,0.017016,0.067521,0.014472,0.027451,0.011753,0.011247,0.022361,0.054199
1,0.029876,0.105808,0.014953,0.017558,0.010855,0.020970,0.014603,0.020567,0.016121,0.019447,0.023879,0.024578,0.033095,0.024476,0.015604,0.047480,0.154025,0.035966,0.071292,0.013566,0.023584,0.031429,0.015793,0.060330,0.013575,0.025139,0.010793,0.008992,0.025865,0.069781
2,0.042637,0.089993,0.018745,0.014072,0.011415,0.021520,0.016180,0.020600,0.016186,0.012538,0.033193,0.021981,0.039027,0.025663,0.013635,0.043400,0.137569,0.038397,0.064716,0.016186,0.023958,0.033620,0.018616,0.073781,0.020728,0.034055,0.008836,0.010188,0.017991,0.060574
3,0.032842,0.159471,0.017090,0.022189,0.013670,0.020169,0.014613,0.017903,0.013779,0.009020,0.027917,0.014774,0.033562,0.010656,0.021021,0.077997,0.160430,0.040344,0.043988,0.014432,0.018923,0.029695,0.015162,0.068178,0.017847,0.027078,0.008553,0.009650,0.014091,0.024961
4,0.025785,0.033486,0.013944,0.008872,0.007692,0.014400,0.009847,0.100938,0.007413,0.007209,0.015116,0.014640,0.019630,0.008774,0.007402,0.013464,0.487447,0.008286,0.015253,0.014354,0.010892,0.019827,0.007898,0.023632,0.004859,0.055392,0.006202,0.019524,0.007319,0.010503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.022528,0.039955,0.016269,0.010084,0.014024,0.009597,0.013329,0.045796,0.011229,0.010015,0.016046,0.014588,0.016959,0.012138,0.009606,0.014919,0.525009,0.010913,0.019390,0.012515,0.013350,0.016923,0.006119,0.023438,0.005253,0.050807,0.009409,0.008070,0.008949,0.012774
24996,0.036257,0.110410,0.025822,0.021428,0.014275,0.038738,0.014351,0.016899,0.014520,0.013363,0.040965,0.024173,0.042290,0.018410,0.018509,0.035063,0.088581,0.033215,0.071423,0.018878,0.037414,0.027527,0.018129,0.078528,0.017435,0.025105,0.015158,0.010909,0.022275,0.049952
24997,0.032884,0.060733,0.020027,0.011159,0.013584,0.014256,0.015359,0.043356,0.019235,0.010682,0.017681,0.024560,0.018604,0.011356,0.011865,0.031049,0.379482,0.015584,0.032400,0.022382,0.015320,0.026363,0.009521,0.039796,0.011585,0.037784,0.009535,0.012426,0.011083,0.020351
24998,0.024319,0.040981,0.010826,0.009396,0.006350,0.016571,0.009178,0.107338,0.009787,0.009756,0.012735,0.023894,0.026807,0.009584,0.006612,0.018054,0.404725,0.015105,0.021737,0.017123,0.012708,0.016423,0.008112,0.023958,0.005657,0.085510,0.006993,0.015575,0.008682,0.015503


In [ ]:
tempPred = tempPred.rename(columns=encoder)
tempPred

,4,5,11,40,55,89,251,374,413,465,701,739,773,803,1003,1024,1045,1049,1052,1084,1186,1194,1213,1251,1254,1295,1409,1590,1687,1724
0,0.023923,0.132491,0.013514,0.019273,0.009867,0.021008,0.015440,0.022611,0.013162,0.014799,0.026106,0.024532,0.038803,0.015088,0.016503,0.062014,0.155990,0.031444,0.054619,0.013503,0.020925,0.028365,0.017016,0.067521,0.014472,0.027451,0.011753,0.011247,0.022361,0.054199
1,0.029876,0.105808,0.014953,0.017558,0.010855,0.020970,0.014603,0.020567,0.016121,0.019447,0.023879,0.024578,0.033095,0.024476,0.015604,0.047480,0.154025,0.035966,0.071292,0.013566,0.023584,0.031429,0.015793,0.060330,0.013575,0.025139,0.010793,0.008992,0.025865,0.069781
2,0.042637,0.089993,0.018745,0.014072,0.011415,0.021520,0.016180,0.020600,0.016186,0.012538,0.033193,0.021981,0.039027,0.025663,0.013635,0.043400,0.137569,0.038397,0.064716,0.016186,0.023958,0.033620,0.018616,0.073781,0.020728,0.034055,0.008836,0.010188,0.017991,0.060574
3,0.032842,0.159471,0.017090,0.022189,0.013670,0.020169,0.014613,0.017903,0.013779,0.009020,0.027917,0.014774,0.033562,0.010656,0.021021,0.077997,0.160430,0.040344,0.043988,0.014432,0.018923,0.029695,0.015162,0.068178,0.017847,0.027078,0.008553,0.009650,0.014091,0.024961
4,0.025785,0.033486,0.013944,0.008872,0.007692,0.014400,0.009847,0.100938,0.007413,0.007209,0.015116,0.014640,0.019630,0.008774,0.007402,0.013464,0.487447,0.008286,0.015253,0.014354,0.010892,0.019827,0.007898,0.023632,0.004859,0.055392,0.006202,0.019524,0.007319,0.010503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.022528,0.039955,0.016269,0.010084,0.014024,0.009597,0.013329,0.045796,0.011229,0.010015,0.016046,0.014588,0.016959,0.012138,0.009606,0.014919,0.525009,0.010913,0.019390,0.012515,0.013350,0.016923,0.006119,0.023438,0.005253,0.050807,0.009409,0.008070,0.008949,0.012774
24996,0.036257,0.110410,0.025822,0.021428,0.014275,0.038738,0.014351,0.016899,0.014520,0.013363,0.040965,0.024173,0.042290,0.018410,0.018509,0.035063,0.088581,0.033215,0.071423,0.018878,0.037414,0.027527,0.018129,0.078528,0.017435,0.025105,0.015158,0.010909,0.022275,0.049952
24997,0.032884,0.060733,0.020027,0.011159,0.013584,0.014256,0.015359,0.043356,0.019235,0.010682,0.017681,0.024560,0.018604,0.011356,0.011865,0.031049,0.379482,0.015584,0.032400,0.022382,0.015320,0.026363,0.009521,0.039796,0.011585,0.037784,0.009535,0.012426,0.011083,0.020351
24998,0.024319,0.040981,0.010826,0.009396,0.006350,0.016571,0.009178,0.107338,0.009787,0.009756,0.012735,0.023894,0.026807,0.009584,0.006612,0.018054,0.404725,0.015105,0.021737,0.017123,0.012708,0.016423,0.008112,0.023958,0.005657,0.085510,0.006993,0.015575,0.008682,0.015503


In [ ]:
print(max(tempPred.iloc[:1].values.flatten()))
print(list(tempPred.iloc[:1].values.flatten()).index(max(tempPred.iloc[:1].values.flatten())))
print(encoder[list(tempPred.iloc[:1].values.flatten()).index(max(tempPred.iloc[:1].values.flatten()))])

0.15599021
16
1045


In [ ]:
model.evaluate([X_valid_ids, X_valid_attention], np.array(pd.get_dummies(y_valid)))

782/782 [==============================] - 32s 41ms/step - loss: 2.4093 - accuracy: 0.2853


[2.4092679023742676, 0.2853200137615204]